In [100]:
import pandas as pd
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
import pickle, json
from urllib import request, parse
from bs4 import BeautifulSoup
from tqdm import tqdm
from selenium import webdriver
from time import sleep
import time, timeit
from selenium.webdriver.common.keys import Keys

In [101]:
#챔피언 id/이름 목록 파일 불러오기
with open('champion_id_and_name_list.pickle','rb') as f1:
    champ_id_dict=pickle.load(f1)

In [102]:
driver=  webdriver.Chrome()

In [110]:
def parse_tup(summonerName, champion, recent=False, champW=False): #동원님 스크립트를 이용해서 함수 재조합.
    global driver, champ_id_dict
    name_parsed = parse.quote_plus(summonerName)
    if str(champion).isnumeric():
        champ = champ_id_dict[champion] #id -챔피언명 정보 딕셔너리로 불러와주세요. 딕셔너리 이름 다르게 했으면 이 이름으로 하셔야 합니다. 
    else: #영어면 
        champ = champion
    driver.get(f'https://www.op.gg/summoner/userName={name_parsed}') #url 오류
    if not(recent):
        try:
            
            driver.find_element_by_xpath('//*[@id="right_gametype_soloranked"]/a').click()
            sleep(2)
            try:
                recentW_R = int(driver.find_element_by_css_selector('div.WinRatioGraph>div.Text').get_attribute('innerText').replace('%',""))
            except:
                recentW_R = -1
            try:
                recentKDA= float(driver.find_element_by_css_selector('span.KDARatio').get_attribute('innerText').split(':')[0])
            except:
                recentKDA = -1
            try:
                totalW_R=int(driver.find_element_by_css_selector('span.winratio').get_attribute('innerText').split()[-1][:-1])
            except:
                totalW_R = -1
        except:
            recentW_R = 0
            recentKDA = 0
            totalW_R = 0
    else: #recent 데이터가 있으면
        recentW_R = recent[0]
        recentKDA = recent[1]
        totalW_R = recent[2]
    sleep(2)
    
    sleep(2)
    driver.find_element_by_xpath('//*[@id="left_champion"]/a').click()
    sleep(3)
    driver.find_element_by_xpath('//*[@id="champion_season"]/li[2]/a').click()
    sleep(5)
    tables = driver.find_elements_by_tag_name('table')
    trs=tables[-1].find_elements_by_xpath('./tbody/tr')
    #픽 champion의 시즌 승률 검색 만약 데이터가 없으면 최근승률로 대체
    if not(champW):
        for tr in trs:
            tds = tr.find_elements_by_xpath('./td')
            if tds[2].get_attribute('data-value')==champ:
                try:
                    chamW_R=round(float(tds[3].get_attribute('data-value')))
                except:
                    chamW_R = -1
                try:
                    chamKDA=float(tds[4].get_attribute("data-value"))
                except:
                    chamKDA = -1
                try:
                    try:
                        matchCNT1=int(tds[3].find_element_by_css_selector('div.Text.Left').get_attribute('innerText')[:-1])
                    except:
                        matchCNT1 = 0
                    try:
                        matchCNT2 = int(tds[3].find_element_by_css_selector('div.Text.Right').get_attribute('innerText')[:-1])
                    except:
                        matchCNT2 = 0
                    matchCNT = matchCNT1 + matchCNT2
                except: #안 되면 Xpath 방식으로 긁어와보자.
                    try:
                        matchCNT1 = int(tds[3].find_element_by_xpath(',/div/div/div[2]').get_attribute('innerText')[:-1])
                        try:
                            matchCNT2 = int(tds[3].find_element_by_xpath(',/div/div/div[4]').get_attribute('innerText')[:-1])
                        except:
                            matchCNT2 = 0
                        matchCNT = matchCNT1 + matchCNT2
                    except:
                        matchCNT = -1
                break
        else:
            chamW_R=recentW_R
            chamKDA=recentKDA
            matchCNT = 0
        sleep(2)
    else:
        chamW_R = champW[0]
        chamKDA = champW[1]
        matchCNT = champW[2]
        

    return [summonerName,champ,chamW_R,chamKDA,recentW_R,recentKDA, totalW_R ,matchCNT]
    
    

In [115]:
#잘 작동하는 지 확인하기
driver= webdriver.Chrome() #이번엔 옵션 없애기...
driver.get('https://op.gg/')
driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
sleep(2)
lang_list = driver.find_elements_by_class_name('setting-list__item')
for lang in lang_list:
    if "English" in lang.text:
        lang.click()
sleep(2)
driver.find_elements_by_class_name('setting__button')[0].click()
sleep(2)

print(parse_tup('이즈아펠케틀애쉬', 'Aphelios', False, False))
driver.quit()

['이즈아펠케틀애쉬', 'Aphelios', 53, 2.63, 45, 2.37, 50, 284]


In [118]:
dat_result = []
with open('Gameinfo_with_user_48336_to_60000_all.pickle', 'rb') as f:
    dat_result += pickle.load(f)

In [105]:
dat_result[0:5]

[{'gameId': 4709111969,
  'teamId': 200,
  'win': False,
  'participants': [['뱅 배준식', 'Camille', 50, 1.95, 57, 2.52, 51, 12],
   ['짜 룽', 'Lee Sin', 51, 3.56, 57, 2.5, 51, 632],
   ['happy   game ', 'Yone', 48, 1.34, 48, 1.34, 52, 0],
   ['잠 이 덜 깨다', 'Caitlyn', 44, 2.2, 44, 2.2, 51, 0],
   ['ha1 m1an bao bao', 'Taric', 61, 3.95, 50, 1.72, 54, 341]]},
 {'gameId': 4758157523,
  'teamId': 100,
  'win': False,
  'participants': [['최함몰', 'Galio', 69, 3.19, 41, 2.76, 57, 52],
   ['라인양보좀부캐임', 'Lee Sin', 25, 2.26, 44, 2.09, 52, 12],
   ['향 유', 'Wukong', 54, 2.24, 48, 2.53, 50, 24],
   ['날씨요정 정현', 'Ashe', 51, 2.79, 52, 3.16, 51, 59],
   ['Navy SEAL 이근', 'Pantheon', 75, 2.92, 44, 1.85, 53, 40]]},
 {'gameId': 4757526203,
  'teamId': 100,
  'win': True,
  'participants': [['최함몰', 'Galio', 69, 3.19, 41, 2.76, 57, 52],
   ['후루꾸우루꾸', 'Shen', 57, 2.58, 53, 2.35, 52, 523],
   ['Zl존Jungle러', 'Graves', 45, 2.67, 52, 3.08, 50, 55],
   ['심심하면놀러와', 'Lulu', 57, 4.13, 57, 4.13, 57, 0],
   ['이즈아펠케틀애쉬', 'Aphelio

In [106]:
participants = dat_result[0]['participants']

In [107]:
for j in range(len(participants)): #길이가 
    part_data = participants[j]
    champW = part_data[-6:-4]+part_data[-1:] if part_data[-1] !=0 else False # False 조건 
    print(champW)

[50, 1.95, 12]
[51, 3.56, 632]
False
False
[61, 3.95, 341]


In [ ]:
driver= webdriver.Chrome() #우선 영어로 지정.
driver.get('https://op.gg/')
driver.find_element_by_xpath('/html/body/div[2]/header/div[2]/div/div/div/div/div/button').click()
sleep(2)
lang_list = driver.find_elements_by_class_name('setting-list__item')
for lang in lang_list:
    if "English" in lang.text:
        lang.click()
sleep(2)
driver.find_elements_by_class_name('setting__button')[0].click()
sleep(2)

for dat in dat_result:
    sleep(2)
    participants = dat['participants'] # participant data 가져오기
    for j in range(len(participants)): #길이가 
        part_data = participants[j]
        print(part_data)
        if part_data[-1] == 0:
            champW = False
        else:
            champW = part_data[-6:-4]+part_data[-1:]
        if champW == False:
            try: #없을 수도 있으니 
                part_data = parse_tup(part_data[0], part_data[1], part_data[-4:-1], champW)
                participants[j] = part_data
            except:
                continue
        sleep(1)
    dat['participants'] = participants
    print(participants)

driver.quit()
    

['뱅 배준식', 'Camille', 50, 1.95, 57, 2.52, 51, 12]
['짜 룽', 'Lee Sin', 51, 3.56, 57, 2.5, 51, 632]
['happy   game ', 'Yone', 48, 1.34, 48, 1.34, 52, 0]
['잠 이 덜 깨다', 'Caitlyn', 44, 2.2, 44, 2.2, 51, 0]
['ha1 m1an bao bao', 'Taric', 61, 3.95, 50, 1.72, 54, 341]
[['뱅 배준식', 'Camille', 50, 1.95, 57, 2.52, 51, 12], ['짜 룽', 'Lee Sin', 51, 3.56, 57, 2.5, 51, 632], ['happy   game ', 'Yone', 40, 1.62, 48, 1.34, 52, 10], ['잠 이 덜 깨다', 'Caitlyn', 57, 2.4, 44, 2.2, 51, 93], ['ha1 m1an bao bao', 'Taric', 61, 3.95, 50, 1.72, 54, 341]]
['최함몰', 'Galio', 69, 3.19, 41, 2.76, 57, 52]
['라인양보좀부캐임', 'Lee Sin', 25, 2.26, 44, 2.09, 52, 12]
['향 유', 'Wukong', 54, 2.24, 48, 2.53, 50, 24]
['날씨요정 정현', 'Ashe', 51, 2.79, 52, 3.16, 51, 59]
['Navy SEAL 이근', 'Pantheon', 75, 2.92, 44, 1.85, 53, 40]
[['최함몰', 'Galio', 69, 3.19, 41, 2.76, 57, 52], ['라인양보좀부캐임', 'Lee Sin', 25, 2.26, 44, 2.09, 52, 12], ['향 유', 'Wukong', 54, 2.24, 48, 2.53, 50, 24], ['날씨요정 정현', 'Ashe', 51, 2.79, 52, 3.16, 51, 59], ['Navy SEAL 이근', 'Pantheon', 75, 2

['bot no', 'Lucian', 47, 2.71, 50, 3.95, 51, 91]
['작은꼬마여우', 'Thresh', 58, 3.1, 59, 3.99, 53, 113]
[['넌너무감정적이야', 'Orianna', 53, 3.13, 41, 2.43, 50, 66], ['최함몰', 'Sylas', 63, 2.51, 41, 2.76, 57, 51], ['토리초리', 'Kayn', 65, 3.47, 53, 2.7, 52, 23], ['bot no', 'Lucian', 47, 2.71, 50, 3.95, 51, 91], ['작은꼬마여우', 'Thresh', 58, 3.1, 59, 3.99, 53, 113]]
['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40]
['토이 콤플렉스', 'Pyke', 50, 2.77, 53, 3.21, 51, 175]
['김해 마이', 'Master Yi', 54, 2.76, 45, 2.11, 51, 677]
['종 서', 'Jhin', 54, 2.6, 46, 1.43, 52, 50]
['Loneliest Girl', 'Sylas', 38, 1.49, 34, 1.73, 51, 108]
[['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40], ['토이 콤플렉스', 'Pyke', 50, 2.77, 53, 3.21, 51, 175], ['김해 마이', 'Master Yi', 54, 2.76, 45, 2.11, 51, 677], ['종 서', 'Jhin', 54, 2.6, 46, 1.43, 52, 50], ['Loneliest Girl', 'Sylas', 38, 1.49, 34, 1.73, 51, 108]]
['Alphabet N', 'Gangplank', 55, 2.87, 42, 1.74, 49, 163]
['adad00ad', 'Orianna', 47, 3.26, 47, 3.26, 50, 0]
['jungukb', 'Lee Sin', 34, 2.1, 56, 1.87, 49,

['Loneliest Girl', 'Jax', 58, 2.24, 34, 1.73, 51, 1490]
['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40]
['청순한 주황버섯', 'Lux', 59, 3.61, 55, 2.86, 52, 54]
['Nigror', 'Lillia', 54, 3.52, 60, 2.59, 50, 74]
['Traho', 'Caitlyn', 58, 3.33, 52, 3.75, 51, 113]
[['Loneliest Girl', 'Jax', 58, 2.24, 34, 1.73, 51, 1490], ['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40], ['청순한 주황버섯', 'Lux', 59, 3.61, 55, 2.86, 52, 54], ['Nigror', 'Lillia', 54, 3.52, 60, 2.59, 50, 74], ['Traho', 'Caitlyn', 58, 3.33, 52, 3.75, 51, 113]]
['yuyuseong', 'Morgana', 62, 4.29, 62, 4.29, 51, 0]
['Lv 1 세구리', 'Camille', 50, 2.14, 47, 2.83, 54, 74]
['원딜이죽지않아', 'Twisted Fate', 45, 3.21, 45, 3.21, 51, 0]
['정신병있는베인', 'Vayne', 54, 1.89, 39, 2.38, 54, 231]
['god호신', 'Nidalee', 35, 3.11, 59, 2.32, 55, 17]
[['yuyuseong', 'Morgana', 49, 2.36, 62, 4.29, 51, 274], ['Lv 1 세구리', 'Camille', 50, 2.14, 47, 2.83, 54, 74], ['원딜이죽지않아', 'Twisted Fate', 33, 3.75, 45, 3.21, 51, 3], ['정신병있는베인', 'Vayne', 54, 1.89, 39, 2.38, 54, 231], ['god호신', 'Nidalee

['DRX 080', 'Jhin', 44, 2.01, 44, 2.01, 62, 0]
[['숨결을 만지던 밤', 'Akali', 68, 2.34, 46, 1.8, 61, 22], ['섹시한 핫세', 'Irelia', 52, 1.74, 54, 3.91, 50, 33], ['룰루 바드 잔나', 'Elise', 67, 2.15, 45, 1.99, 49, 3], ['에밀리 다이어', 'Seraphine', 60, 2.33, 50, 2.95, 50, 5], ['DRX 080', 'Jhin', 44, 2.01, 44, 2.01, 62, 0]]
['ADout CarRy', 'Lucian', 43, 2.43, 43, 2.43, 49, 0]
['단검이 어디있지', 'Camille', 44, 2.46, 44, 2.46, 52, 0]
['안전밸트꼭매라', 'Rammus', 52, 2.34, 52, 2.34, 51, 0]
['메라병 그랩', 'Nami', 53, 4.26, 53, 4.26, 70, 0]
['하은이 미드', 'Anivia', 49, 2.21, 49, 2.21, 52, 0]
[['ADout CarRy', 'Lucian', 40, 1.98, 43, 2.43, 49, 30], ['단검이 어디있지', 'Camille', 44, 2.46, 44, 2.46, 52, 0], ['안전밸트꼭매라', 'Rammus', 52, 2.34, 52, 2.34, 51, 0], ['메라병 그랩', 'Nami', 53, 4.26, 53, 4.26, 70, 0], ['하은이 미드', 'Anivia', 49, 2.21, 49, 2.21, 52, 0]]
['AfreecaTV 이인수', 'LeBlanc', 67, 2.62, 51, 2.53, 52, 9]
['살려주시오제발', 'Kindred', 50, 3.08, 53, 3.66, 55, 2]
['우한폐렴인데요', 'Ashe', 50, 2.16, 45, 1.82, 50, 4]
['전략적버러지', 'Seraphine', 50, 4.15, 50, 4.15, 60

['더샤이3', 'Sylas', 36, 2.0, 45, 2.29, 35, 11]
['v와드싸개v', 'Senna', 55, 3.13, 46, 2.0, 45, 22]
['철신지옥의이꾸', 'Amumu', 50, 1.92, 50, 1.92, 43, 0]
['코칼리', 'Aphelios', 43, 2.14, 57, 2.91, 52, 14]
['빠가라', 'Garen', 50, 3.2, 50, 3.2, 52, 0]
[['더샤이3', 'Sylas', 36, 2.0, 45, 2.29, 35, 11], ['v와드싸개v', 'Senna', 55, 3.13, 46, 2.0, 45, 22], ['철신지옥의이꾸', 'Amumu', 50, 1.92, 50, 1.92, 43, 0], ['코칼리', 'Aphelios', 43, 2.14, 57, 2.91, 52, 14], ['빠가라', 'Garen', 58, 3.56, 50, 3.2, 52, 52]]
['adventure hunter', 'Kindred', 57, 2.6, 52, 2.1, 54, 556]
['엄딩이 조아', 'Volibear', 25, 1.4, 48, 3.06, 56, 12]
['한계일때 한개 더', 'Zoe', 57, 3.67, 46, 2.06, 52, 49]
['Sniper Gun', 'Pantheon', 20, 1.83, 43, 1.97, 52, 5]
['Ptolemaeos', 'Jhin', 51, 2.39, 52, 2.23, 51, 84]
[['adventure hunter', 'Kindred', 57, 2.6, 52, 2.1, 54, 556], ['엄딩이 조아', 'Volibear', 25, 1.4, 48, 3.06, 56, 12], ['한계일때 한개 더', 'Zoe', 57, 3.67, 46, 2.06, 52, 49], ['Sniper Gun', 'Pantheon', 20, 1.83, 43, 1.97, 52, 5], ['Ptolemaeos', 'Jhin', 51, 2.39, 52, 2.23, 51, 84]]


...

['괴물 백지민', 'Jhin', 47, 2.52, 51, 3.89, 51, 30]
[['LS AsuraBalBaltA', 'Trundle', 50, 2.83, 56, 2.88, 51, 8], ['Padarg', 'Thresh', 54, 3.06, 53, 1.92, 54, 342], ['지느머리엉겅퀴', 'Katarina', 46, 1.84, 57, 2.98, 51, 13], ['모유를바쳐라', 'Singed', 56, 2.43, 45, 1.75, 51, 563], ['괴물 백지민', 'Jhin', 47, 2.52, 51, 3.89, 51, 30]]
['LS AsuraBalBaltA', "Kha'Zix", 61, 2.5, 56, 2.88, 51, 18]
['tough game', 'Senna', 35, 1.67, 35, 1.67, 49, 0]
['  Rvers', 'Twisted Fate', 50, 2.33, 45, 2.01, 54, 2]
['Y A O S', "Vel'Koz", 41, 2.44, 40, 1.6, 53, 27]
['jing shen bingG', 'Renekton', 44, 1.79, 44, 1.79, 53, 0]
[['LS AsuraBalBaltA', "Kha'Zix", 61, 2.5, 56, 2.88, 51, 18], ['tough game', 'Senna', 35, 1.67, 35, 1.67, 49, 0], ['  Rvers', 'Twisted Fate', 50, 2.33, 45, 2.01, 54, 2], ['Y A O S', "Vel'Koz", 41, 2.44, 40, 1.6, 53, 27], ['jing shen bingG', 'Renekton', 44, 1.79, 44, 1.79, 53, 0]]
['LS AsuraBalBaltA', 'Master Yi', 33, 1.53, 56, 2.88, 51, 3]
['우린남이니까남탓을할수도있다', 'Sett', 33, 4.0, 42, 2.35, 57, 3]
['박종우도파페이커', 'Aurelio

[['ephemearl', 'Camille', 76, 3.73, 46, 2.42, 63, 76], ['I cant laugh', 'Elise', 67, 4.29, 60, 2.45, 65, 3], ['Massless', 'Bard', 54, 2.94, 58, 3.03, 51, 441], ['평범한 숟가락', 'Samira', 57, 2.49, 51, 2.61, 53, 110], ['     9TvT6', 'Jhin', 50, 3.49, 50, 3.49, 51, 0]]
['학수고대하다', 'Senna', 43, 2.62, 40, 1.78, 51, 46]
['강진관', 'Twisted Fate', 64, 3.36, 47, 2.34, 64, 135]
['HJ Fanboy', 'Graves', 62, 4.09, 64, 3.68, 64, 21]
['지존정글짱짱', 'Leona', 57, 2.54, 60, 2.31, 53, 7]
['충주갱수', 'Camille', 39, 2.82, 39, 2.82, 51, 0]
[['학수고대하다', 'Senna', 43, 2.62, 40, 1.78, 51, 46], ['강진관', 'Twisted Fate', 64, 3.36, 47, 2.34, 64, 135], ['HJ Fanboy', 'Graves', 62, 4.09, 64, 3.68, 64, 21], ['지존정글짱짱', 'Leona', 57, 2.54, 60, 2.31, 53, 7], ['충주갱수', 'Camille', 53, 1.98, 39, 2.82, 51, 172]]
['January 31', 'Maokai', 47, 2.31, 47, 2.31, 53, 0]
['사람이름이함민기', "Kai'Sa", 43, 1.91, 43, 1.91, 52, 0]
['율이맨탈은유리', 'Aatrox', 63, 2.57, 49, 1.99, 57, 139]
['H4VEN', 'Talon', 60, 3.38, 53, 3.47, 68, 10]
['all mute 수고', 'Sejuani', 67, 4.4,

['BD jjjellyfish', 'Camille', 44, 1.74, 50, 2.55, 53, 66]
['오소리와너구리', 'Nocturne', 54, 2.55, 50, 2.24, 53, 903]
['20old  all ', 'Evelynn', 50, 1.95, 57, 2.34, 57, 4]
['청희야', "Kog'Maw", 60, 2.55, 54, 1.71, 59, 370]
['룰루게임', 'Lulu', 61, 4.31, 59, 3.09, 59, 242]
[['BD jjjellyfish', 'Camille', 44, 1.74, 50, 2.55, 53, 66], ['오소리와너구리', 'Nocturne', 54, 2.55, 50, 2.24, 53, 903], ['20old  all ', 'Evelynn', 50, 1.95, 57, 2.34, 57, 4], ['청희야', "Kog'Maw", 60, 2.55, 54, 1.71, 59, 370], ['룰루게임', 'Lulu', 61, 4.31, 59, 3.09, 59, 242]]
['AD WEAPON', 'Xayah', 51, 2.42, 51, 2.42, 51, 0]
['미친듯이 솔킬', 'Zac', 43, 3.24, 46, 1.77, 50, 7]
['JUST SAVIOR', 'Twisted Fate', 52, 2.61, 52, 2.61, 53, 0]
['이프로 지킴이', 'Bard', 47, 2.32, 47, 2.32, 50, 0]
['미친듯이 이긴다', 'Fiora', 50, 2.84, 50, 2.84, 58, 0]
[['AD WEAPON', 'Xayah', 50, 2.49, 51, 2.42, 51, 8], ['미친듯이 솔킬', 'Zac', 43, 3.24, 46, 1.77, 50, 7], ['JUST SAVIOR', 'Twisted Fate', 33, 1.67, 52, 2.61, 53, 3], ['이프로 지킴이', 'Bard', 0, 1.71, 47, 2.32, 50, 2], ['미친듯이 이긴다', 'Fiora

[['똥참기대회탈락', 'Lillia', 46, 3.85, 41, 2.7, 50, 13], ['김재석', 'Ashe', 53, 2.46, 43, 2.18, 51, 148], ['DoubleAiM', 'Akali', 57, 3.15, 43, 2.28, 53, 14], ['LLLL XXXX', 'Renekton', 46, 2.19, 56, 3.23, 49, 98], ['산소생명체', 'Senna', 48, 3.24, 49, 2.59, 55, 60]]
['니쥬깅', 'Syndra', 33, 2.14, 53, 2.09, 52, 9]
['티아트의 대지', 'Udyr', 33, 1.65, 53, 2.52, 48, 3]
['씹 쌍 땃쥐', "Rek'Sai", 54, 3.04, 52, 3.41, 52, 815]
['밤새도록죽여놓는단마인드', "Kai'Sa", 48, 2.43, 48, 2.43, 56, 0]
['수영장파티박경민', 'Blitzcrank', 58, 3.03, 45, 2.87, 50, 48]
[['니쥬깅', 'Syndra', 33, 2.14, 53, 2.09, 52, 9], ['티아트의 대지', 'Udyr', 33, 1.65, 53, 2.52, 48, 3], ['씹 쌍 땃쥐', "Rek'Sai", 54, 3.04, 52, 3.41, 52, 815], ['밤새도록죽여놓는단마인드', "Kai'Sa", 57, 2.75, 48, 2.43, 56, 130], ['수영장파티박경민', 'Blitzcrank', 58, 3.03, 45, 2.87, 50, 48]]
['풀뜯는원딜', 'Jhin', 52, 5.13, 52, 5.13, 65, 0]
['Ggame', 'Gangplank', 80, 4.24, 49, 2.28, 51, 25]
['귤이맛나', 'Thresh', 38, 1.92, 55, 1.93, 50, 16]
['닭갈비치킨', 'Ahri', 50, 2.94, 54, 2.94, 55, 4]
['콩사령관', "Rek'Sai", 66, 3.8, 55, 3.03, 56, 91]
[

['I Love YeonHee', 'Thresh', 48, 3.06, 50, 2.55, 51, 345]
['credibl2', 'Ornn', 51, 2.97, 46, 3.11, 52, 167]
['사람이름이함민기', "Kai'Sa", 43, 1.91, 43, 1.91, 52, 0]
['레 큐엠', 'Sylas', 48, 1.95, 38, 1.15, 51, 107]
[['성준 S2', 'Graves', 59, 3.65, 56, 3.66, 52, 44], ['I Love YeonHee', 'Thresh', 48, 3.06, 50, 2.55, 51, 345], ['credibl2', 'Ornn', 51, 2.97, 46, 3.11, 52, 167], ['사람이름이함민기', "Kai'Sa", 41, 2.07, 43, 1.91, 52, 29], ['레 큐엠', 'Sylas', 48, 1.95, 38, 1.15, 51, 107]]
['Forever plunder', 'Garen', 51, 2.51, 51, 2.51, 51, 0]
['SN sofham', 'Pantheon', 57, 2.7, 50, 2.16, 51, 21]
['말랑무', 'Tristana', 58, 3.95, 53, 3.55, 73, 19]
['재나죽으면탈주', 'Zed', 50, 2.5, 55, 2.53, 56, 10]
['IYl', 'Evelynn', 55, 2.47, 57, 2.93, 51, 814]
[['Forever plunder', 'Garen', 42, 1.9, 51, 2.51, 51, 156], ['SN sofham', 'Pantheon', 57, 2.7, 50, 2.16, 51, 21], ['말랑무', 'Tristana', 58, 3.95, 53, 3.55, 73, 19], ['재나죽으면탈주', 'Zed', 50, 2.5, 55, 2.53, 56, 10], ['IYl', 'Evelynn', 55, 2.47, 57, 2.93, 51, 814]]
['겸손하면복이온다', 'Jhin', 51, 4

In [117]:
dat_result[5:10]

[{'gameId': 4756414494,
  'teamId': 200,
  'win': True,
  'participants': [['최함몰', 'Galio', 69, 3.19, 41, 2.76, 57, 52],
   ['3렙갱안감', 'Nidalee', 60, 3.68, 55, 2.59, 59, 105],
   ['피카츄라이츄파이리꼬북이', 'Aphelios', 52, 2.31, 50, 2.69, 52, 44],
   ['the night alight', 'Jax', 70, 2.48, 43, 2.3, 55, 30],
   ['미사카 미코토', 'Lulu', 43, 3.47, 50, 4.23, 52, 69]]},
 {'gameId': 4755642201,
  'teamId': 200,
  'win': True,
  'participants': [['Love IU Forever', 'Sylas', 54, 2.31, 38, 1.45, 53, 57],
   ['최함몰', 'Renekton', 60, 1.79, 41, 2.76, 57, 40],
   ['춥 화 춥 스', 'Jhin', 62, 3.53, 58, 2.85, 53, 21],
   ['규용띠용', 'Bard', 53, 2.52, 56, 4.74, 50, 96],
   ['Klxg', 'Graves', 40, 4.07, 58, 2.79, 48, 5]]},
 {'gameId': 4755407845,
  'teamId': 100,
  'win': False,
  'participants': [['인간말종망한인생', 'Jhin', 62, 3.24, 50, 3.13, 52, 42],
   ['내 인생이 다딱이', 'Gangplank', 25, 1.92, 40, 1.63, 54, 12],
   ['햄토리는자유에요', 'Leona', 53, 2.35, 46, 2.36, 52, 95],
   ['오로치먀루', 'Cassiopeia', 59, 2.62, 48, 3.82, 52, 429],
   ['DickSpicy', 

In [ ]:
dat_result = []
with open('Gameinfo_with_user_110000_to_120000_all.pickle', 'rb') as g:
    dat_result += pickle.load(g)